In [9]:
import pandas as pd

df = pd.read_csv('20251031_Nacional.csv')

In [10]:
df['year'] = df['year'].astype(int)
df['quarter'] = df['quarter'].astype(int)
df

,periodo,year,quarter,pob_total,pob_15_y_mas,pob_hombres_total,pob_mujeres_total,pea_total,pea_hombres,pea_mujeres,...,pnea_no_disponible,subordinados_remunerados,empleadores,cuenta_propia,trabajadores_no_remunerados,ocupacion_formal,ocupacion_informal,subocupacion,ing_prom_mes_total,ing_prom_hora_total
0,2005.0-T1.0,2005,1,104358830.0,71466330.0,50417692.0,53941138.0,42106336.0,26989423.0,15116913.0,...,25353379.0,26076769.0,1904995.0,9620836.0,2884471.0,16485562.0,24001509.0,3610838.0,4132.146780,24.644910
1,2005.0-T2.0,2005,2,104717538.0,71761366.0,50254331.0,54463207.0,42219007.0,26738081.0,15480926.0,...,25231842.0,26168794.0,1927462.0,9729316.0,2927412.0,16486775.0,24266209.0,3076093.0,4149.712531,24.189184
2,2005.0-T3.0,2005,3,105077468.0,72123553.0,50433685.0,54643783.0,43047402.0,27171531.0,15875871.0,...,24765570.0,26370444.0,1948243.0,9957927.0,3148747.0,16612721.0,24812640.0,3027619.0,4216.083220,24.618803
3,2005.0-T4.0,2005,4,105439074.0,72614218.0,50612869.0,54826205.0,43314284.0,27268601.0,16045683.0,...,24909299.0,27142075.0,1977647.0,9894574.0,2951702.0,16916585.0,25049413.0,2670581.0,4320.199511,25.201129
4,2006.0-T1.0,2006,1,105802069.0,72836619.0,50775004.0,55027065.0,43400348.0,27336757.0,16063591.0,...,24894630.0,27339282.0,2032277.0,9753273.0,2745443.0,17288778.0,24581497.0,2606593.0,4359.247495,25.623025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2024.0-T2.0,2024,2,129940228.0,101172040.0,61939986.0,68000242.0,60953610.0,36017628.0,24935982.0,...,35289140.0,41034357.0,3190143.0,13002402.0,2100259.0,27136469.0,32190692.0,4378878.0,9550.139269,60.054579
78,2024.0-T3.0,2024,3,130154247.0,101620868.0,62071834.0,68082413.0,61370334.0,36334958.0,25035376.0,...,34973312.0,40982694.0,3167260.0,13158507.0,2219788.0,27003898.0,32524351.0,4784598.0,9758.451379,61.039084
79,2024.0-T4.0,2024,4,130294079.0,101936513.0,62072760.0,68221319.0,61055321.0,35867548.0,25187773.0,...,35398836.0,41362358.0,3447267.0,12689352.0,1988569.0,27096542.0,32391004.0,4889790.0,9849.333992,62.564962
80,2025.0-T1.0,2025,1,130417144.0,102259659.0,62164287.0,68252857.0,60491235.0,35834250.0,24656985.0,...,36508973.0,41004984.0,3043023.0,13023470.0,1929532.0,26958290.0,32042719.0,3902542.0,10164.383808,65.398459


In [11]:
deflactor_mexico = [
    100.00, 100.52, 101.25, 102.34, 103.11, 103.22, 104.13, 105.47,
    106.51, 106.68, 107.82, 109.43, 110.60, 111.64, 113.14, 115.02,
    116.38, 116.44, 117.15, 118.06, 119.92, 120.15, 120.91, 122.37,
    123.36, 123.68, 124.58, 126.11, 127.28, 127.89, 129.23, 131.02,
    132.22, 132.30, 133.22, 134.41, 135.61, 135.83, 136.91, 138.25,
    138.86, 138.76, 139.51, 140.24, 141.01, 141.13, 142.17, 143.94,
    146.12, 147.27, 148.87, 151.10, 153.07, 153.03, 154.51, 156.41,
    157.65, 158.12, 158.83, 159.87, 161.42, 161.27, 162.59, 164.84,
    166.92, 168.12, 170.36, 173.84, 176.63, 179.02, 182.26, 185.73,
    188.76, 188.80, 190.23, 192.51, 195.96, 196.86, 199.11, 201.76,
    203.68, 205.14
]

fechas = pd.period_range(start='2005Q1', end='2025Q2', freq='Q')

df_deflactor = pd.DataFrame({
    'deflactor': deflactor_mexico,
    'fecha_idx': fechas
})

# Extrae 'year' y 'quarter' del índice de fechas
df_deflactor['year'] = df_deflactor['fecha_idx'].dt.year
df_deflactor['quarter'] = df_deflactor['fecha_idx'].dt.quarter


df_deflactor



,deflactor,fecha_idx,year,quarter
0,100.00,2005Q1,2005,1
1,100.52,2005Q2,2005,2
2,101.25,2005Q3,2005,3
3,102.34,2005Q4,2005,4
4,103.11,2006Q1,2006,1
...,...,...,...,...
77,196.86,2024Q2,2024,2
78,199.11,2024Q3,2024,3
79,201.76,2024Q4,2024,4
80,203.68,2025Q1,2025,1


In [12]:
df_final = pd.merge(df, df_deflactor[['year', 'quarter', 'deflactor']], 
                    on=['year', 'quarter'], 
                    how='left')

In [ ]:
#         df_serie_nacional.to_csv(file_name_nacional, index=True)


NameError: name 'Nacional_deflactado' is not defined

In [14]:
# --- 5. Crea las columnas deflactadas (reales) ---
# (Ingreso Nominal / Deflactor) * 100 = Ingreso Real (base 2005)
df_final['ing_prom_mes_real'] = (df_final['ing_prom_mes_total'] / df_final['deflactor']) * 100
df_final['ing_prom_hora_real'] = (df_final['ing_prom_hora_total'] / df_final['deflactor']) * 100

# --- 6. Crea una columna de fecha para graficar ---
# Convertimos 'year' y 'quarter' en una fecha de fin de trimestre
df_final['fecha'] = pd.to_datetime(df_final['year'].astype(str) + 'Q' + df_final['quarter'].astype(str))
df_final['fecha'] = df_final['fecha'] + pd.offsets.QuarterEnd(0)


# --- 7. Exporta los datos a JSON para la página web ---
# Ejecuta este script y copia la salida de la terminal.
# Esa salida la pegarás en el archivo HTML del Paso 3.
print(df_final.to_json(orient='records'))

[{"periodo":"2005.0-T1.0","year":2005,"quarter":1,"pob_total":104358830.0,"pob_15_y_mas":71466330.0,"pob_hombres_total":50417692.0,"pob_mujeres_total":53941138.0,"pea_total":42106336.0,"pea_hombres":26989423.0,"pea_mujeres":15116913.0,"pnea_total":29359994.0,"ocupada_total":40487071.0,"desocupada_total":1619265.0,"pnea_disponible":4006615.0,"pnea_no_disponible":25353379.0,"subordinados_remunerados":26076769.0,"empleadores":1904995.0,"cuenta_propia":9620836.0,"trabajadores_no_remunerados":2884471.0,"ocupacion_formal":16485562.0,"ocupacion_informal":24001509.0,"subocupacion":3610838.0,"ing_prom_mes_total":4132.1467803251,"ing_prom_hora_total":24.6449097743,"deflactor":100.0,"ing_prom_mes_real":4132.1467803251,"ing_prom_hora_real":24.6449097743,"fecha":1112227200000},{"periodo":"2005.0-T2.0","year":2005,"quarter":2,"pob_total":104717538.0,"pob_15_y_mas":71761366.0,"pob_hombres_total":50254331.0,"pob_mujeres_total":54463207.0,"pea_total":42219007.0,"pea_hombres":26738081.0,"pea_mujeres":15

C:\Users\rodri\AppData\Local\Temp\ipykernel_15080\3641362559.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_final['fecha'] = pd.to_datetime(df_final['year'].astype(str) + 'Q' + df_final['quarter'].astype(str))


In [16]:
df_final.to_csv("Nacional_deflactado.csv", index=True)


In [15]:
df_final

,periodo,year,quarter,pob_total,pob_15_y_mas,pob_hombres_total,pob_mujeres_total,pea_total,pea_hombres,pea_mujeres,...,trabajadores_no_remunerados,ocupacion_formal,ocupacion_informal,subocupacion,ing_prom_mes_total,ing_prom_hora_total,deflactor,ing_prom_mes_real,ing_prom_hora_real,fecha
0,2005.0-T1.0,2005,1,104358830.0,71466330.0,50417692.0,53941138.0,42106336.0,26989423.0,15116913.0,...,2884471.0,16485562.0,24001509.0,3610838.0,4132.146780,24.644910,100.00,4132.146780,24.644910,2005-03-31
1,2005.0-T2.0,2005,2,104717538.0,71761366.0,50254331.0,54463207.0,42219007.0,26738081.0,15480926.0,...,2927412.0,16486775.0,24266209.0,3076093.0,4149.712531,24.189184,100.52,4128.245654,24.064051,2005-06-30
2,2005.0-T3.0,2005,3,105077468.0,72123553.0,50433685.0,54643783.0,43047402.0,27171531.0,15875871.0,...,3148747.0,16612721.0,24812640.0,3027619.0,4216.083220,24.618803,101.25,4164.032810,24.314868,2005-09-30
3,2005.0-T4.0,2005,4,105439074.0,72614218.0,50612869.0,54826205.0,43314284.0,27268601.0,16045683.0,...,2951702.0,16916585.0,25049413.0,2670581.0,4320.199511,25.201129,102.34,4221.418322,24.624907,2005-12-31
4,2006.0-T1.0,2006,1,105802069.0,72836619.0,50775004.0,55027065.0,43400348.0,27336757.0,16063591.0,...,2745443.0,17288778.0,24581497.0,2606593.0,4359.247495,25.623025,103.11,4227.764034,24.850185,2006-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2024.0-T2.0,2024,2,129940228.0,101172040.0,61939986.0,68000242.0,60953610.0,36017628.0,24935982.0,...,2100259.0,27136469.0,32190692.0,4378878.0,9550.139269,60.054579,196.86,4851.234009,30.506237,2024-06-30
78,2024.0-T3.0,2024,3,130154247.0,101620868.0,62071834.0,68082413.0,61370334.0,36334958.0,25035376.0,...,2219788.0,27003898.0,32524351.0,4784598.0,9758.451379,61.039084,199.11,4901.035297,30.655961,2024-09-30
79,2024.0-T4.0,2024,4,130294079.0,101936513.0,62072760.0,68221319.0,61055321.0,35867548.0,25187773.0,...,1988569.0,27096542.0,32391004.0,4889790.0,9849.333992,62.564962,201.76,4881.707966,31.009597,2024-12-31
80,2025.0-T1.0,2025,1,130417144.0,102259659.0,62164287.0,68252857.0,60491235.0,35834250.0,24656985.0,...,1929532.0,26958290.0,32042719.0,3902542.0,10164.383808,65.398459,203.68,4990.369112,32.108434,2025-03-31
